In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Read data for X and y
df7 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/7.csv")
df15 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/15.csv")
df16 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/16.csv")
df21 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/21.csv")
df33 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/33.csv")
df56 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/56.csv")
df68 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/68.csv")
df95 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/95.csv")
df1000 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/1000.csv")
df1001 = pd.read_csv("../Data/Train/Random Samples/5000/Sequences/1001.csv")

df = pd.concat([df7,df15, df16, df21, df33, df56, df68, df95, df1000, df1001], ignore_index=True)

X = df.drop(columns=['y']).values
y = df['y'].values

print(len(X))

28911


In [2]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
sm = SMOTE(k_neighbors=4, random_state=42)
X_res, y_res = sm.fit_resample(X, y)

#oversampler = RandomOverSampler(random_state=42)
#X_res, y_res = oversampler.fit_resample(X,y)

In [17]:
# One-hot-encoding for sequences in X
import numpy as np

list_of_sequences = X

# Create a set of unique IDs
unique_ids = set()
for sequence in list_of_sequences:
    unique_ids.update(sequence)

# Convert the set to a sorted list
sorted_unique_ids = sorted(unique_ids)

# Create a dictionary mapping each ID to its index
id_to_index = {id: index for index, id in enumerate(sorted_unique_ids)}

# One-hot encode each sequence separately
encoded_sequences = []
for sequence in list_of_sequences:
    encoded_sequence = np.zeros((len(sequence), len(sorted_unique_ids)), dtype=int)
    for i, id in enumerate(sequence):
        index = id_to_index[id]
        encoded_sequence[i, index] = 1
    encoded_sequences.append(encoded_sequence)

X = encoded_sequences

In [5]:
print(len(unique_ids))

68


In [18]:
# Flatten nested list in order to pass it to the model

X_flattened = np.array([np.array(row).flatten() for row in X])

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, random_state = 42, test_size = 0.33)

In [15]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(criterion = "entropy", max_features=3, min_samples_leaf=1, min_samples_split=3, n_estimators = 80)
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.5641345696623871


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import joblib

model = RandomForestClassifier()

grid_space={'n_estimators':[10,20,30,40,50,60],
              'max_features':[1,3,5,7,9],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3],
              'criterion': ["entropy","log_loss"]
           }

grid = GridSearchCV(model,param_grid=grid_space,cv=2,scoring="f1_micro",verbose=10)

model_grid = grid.fit(X_flattened,y)

Fitting 2 folds for each of 540 candidates, totalling 1080 fits
[CV 1/2; 1/540] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10
[CV 1/2; 1/540] END criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 2/2; 1/540] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10
[CV 2/2; 1/540] END criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=10;, score=nan total time=   0.0s
[CV 1/2; 2/540] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20
[CV 1/2; 2/540] END criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20;, score=nan total time=   0.0s
[CV 2/2; 2/540] START criterion=entropy, max_features=1, min_samples_leaf=1, min_samples_split=1, n_estimators=20
[CV 2/2; 2/540] END criterion=entropy, max_features

C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
360 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\ensemble\_forest.py", line 340, in fit
    self._validate_params()
  File "C:\Users\jkaszyda\.conda\envs\DeepLearning\lib\site-packages\sklearn\base.py", line 600, in _validate_params
    valid

In [21]:
print('Best hyperparameters are: '+str(model_grid.best_params_))
print('Best score is: '+str(model_grid.best_score_))

Best hyperparameters are: {'criterion': 'log_loss', 'max_features': 1, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 60}
Best score is: 0.525785410306244


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model.classes_)
disp.plot()
plt.show()